<a href="https://colab.research.google.com/github/elodie0778/pirhana/blob/main/goldfish5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# %% [markdown]
# # 1. Mount Google Drive
#
# Mount Google Drive to access and save data persistently.

# %%
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# %% [markdown]
# # 2. Install Necessary Libraries
#
# Install and upgrade necessary libraries. Some libraries might already be installed in Google Colab, but specifying them ensures compatibility.

# %%
# Install necessary libraries
!pip install --upgrade optuna ta catboost
!pip install optuna-integration

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.2 MB/s eta 0:00:00
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=90cdd1fd2991920a88803f6a7a1fa6fc89f5298950d5b681c1445ee2454976ba
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.4/97.4 kB 2.7 MB/s eta 0:00:00


In [3]:
# %%
# Import Libraries
import pandas as pd
import numpy as np
import glob
import os
from datetime import datetime, timedelta

# Machine Learning Libraries
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler

# For Technical Indicators
import ta

# For Optuna Hyperparameter Tuning
import optuna
from optuna.pruners import MedianPruner
from optuna.exceptions import TrialPruned

# For CatBoost
from catboost import CatBoostRegressor, Pool, cv as catboost_cv

# For Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# For saving/loading data
import pickle

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

In [4]:
# %% [markdown]
# # 3. Import Libraries
#
# Import all necessary libraries for data processing, model training, evaluation, and visualization.

# %%
# Import Libraries
import pandas as pd
import numpy as np
import glob
import os
from datetime import datetime, timedelta

# Machine Learning Libraries
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler

# For Technical Indicators
import ta

# For Optuna Hyperparameter Tuning
import optuna
from optuna.pruners import MedianPruner
from optuna.exceptions import TrialPruned

# For CatBoost
from catboost import CatBoostRegressor, Pool, cv as catboost_cv

# For Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# For saving/loading data
import pickle

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

In [5]:
# %% [markdown]
# # 4. Define Paths and Initialize Directories
#
# Define file paths for data storage and ensure necessary directories exist.

# %%
# Define paths
DATA_PATH = '/content/drive/MyDrive/data3/'
STUDIES_DIR = os.path.join(DATA_PATH, 'optuna_studies3')
FORECASTS_PATH = os.path.join(DATA_PATH, 'stock_forecasts3.pkl')
PERFORMANCE_PATH = os.path.join(DATA_PATH, 'model_performance3.pkl')
BEST_PARAMS_PATH = os.path.join(DATA_PATH, 'best_params_dict3.pkl')
FINAL_RANKINGS_PATH = '/content/drive/MyDrive/final_stock_rankings3.csv'

# Create necessary directories if they don't exist
os.makedirs(STUDIES_DIR, exist_ok=True)

In [6]:
# %% [markdown]
# # 5. Load Existing Data
#
# Load existing best parameters, forecasts, and model performance data if available. This ensures continuity in your workflow.

# %%
# Load existing best parameters if available
if os.path.exists(BEST_PARAMS_PATH):
    with open(BEST_PARAMS_PATH, 'rb') as f:
        best_params_dict = pickle.load(f)
    print("Loaded existing best parameters.")
else:
    best_params_dict = {}
    print("No existing best parameters found. Starting fresh.")

# Load existing forecasts if available
if os.path.exists(FORECASTS_PATH):
    with open(FORECASTS_PATH, 'rb') as f:
        stock_forecasts = pickle.load(f)
    print("Loaded existing forecasts.")
else:
    stock_forecasts = {}
    print("No existing forecasts found. Starting fresh.")

# Load existing model performance data if available
if os.path.exists(PERFORMANCE_PATH):
    with open(PERFORMANCE_PATH, 'rb') as f:
        model_performance = pickle.load(f)
    print("Loaded existing model performance data.")
else:
    model_performance = []
    print("No existing model performance data found. Starting fresh.")

Loaded existing best parameters.
Loaded existing forecasts.
Loaded existing model performance data.


In [7]:
# %% [markdown]
# # 6. Load and Preprocess Stock Data
#
# Load all cleaned stock CSV files from the `data3` folder, merge them with `FinalScore` from `final_stock_rankings3.csv`, and discard any stocks that do not have a `FinalScore`.

# %%
# List all CSV files in the directory matching '*_data.csv'
csv_files = glob.glob(os.path.join(DATA_PATH, '*_data.csv'))

print(f"Total CSV files found: {len(csv_files)}")

# Load FinalScore data
if os.path.exists(FINAL_RANKINGS_PATH):
    final_rankings = pd.read_csv(FINAL_RANKINGS_PATH)
    print("Loaded final_stock_rankings3.csv.")
else:
    raise FileNotFoundError("final_stock_rankings3.csv not found in the data3 folder.")

# Keep only 'Ticker' and 'FinalScore'
final_rankings = final_rankings[['Ticker', 'FinalScore']].dropna()
final_rankings = final_rankings.drop_duplicates(subset='Ticker')

# List of tickers with FinalScore
valid_tickers = set(final_rankings['Ticker'].unique())

print(f"Number of valid tickers with FinalScore: {len(valid_tickers)}")

# Initialize a dictionary to store DataFrames
stock_data = {}

# Feature Engineering Function
def add_technical_indicators(df):
    df = df.copy()
    # Since the new data includes various price metrics, additional technical indicators can be minimal
    # However, if needed, you can add more indicators here
    df.dropna(inplace=True)
    return df

# Load and preprocess each CSV file
for file in csv_files:
    ticker = os.path.basename(file).split('_')[0]
    # Only consider tickers that appear in final_rankings
    if ticker not in valid_tickers:
        continue
    df = pd.read_csv(file)
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values('Date')
    df = add_technical_indicators(df)
    df.reset_index(drop=True, inplace=True)
    stock_data[ticker] = df

print(f"Total stocks loaded after filtering: {len(stock_data)}")

Total CSV files found: 282
Loaded final_stock_rankings3.csv.
Number of valid tickers with FinalScore: 210
Total stocks loaded after filtering: 210


In [8]:
# %% [markdown]
# # 7. Merge FinalScore and Additional Preprocessing
#
# Assign the `FinalScore` to each stock's DataFrame and prepare the data for modeling by defining features and the target variable.

# %%
# Initialize a dictionary to store FinalScore for each stock
final_score_dict = final_rankings.set_index('Ticker')['FinalScore'].to_dict()

# Assign FinalScore to each stock's DataFrame
for ticker, df in stock_data.items():
    df['FinalScore'] = final_score_dict.get(ticker, 0)  # Assign 0 if not found, though all tickers should have FinalScore
    stock_data[ticker] = df

# Feature Engineering and Target Definition
feature_cols = ['Market_Cap_EBT_Excl_Unusual_Items', 'Market_Cap_Total_Revenue',
               'Market_Cap_Book_Value', 'Price_Last_Quarter_Levered_FCF',
               'Price_Funds_From_Operations', 'Price_Tangible_Book',
               'TEV_Excl_Opp_Leases_Total_Revenue', 'TEV_Employees',
               'Price_Last Quarter_EPS(SNL)', 'Price_Last_Quarter_Net_FCF(SNL)',
               'Price_Sales(SNL)', 'Price_Adj_Funds_From_Opperations',
               'Price_Forward_EPS', 'PEG_Ratio',
               'Market_Cap_Forward_Total_Revenue',
               'Diluted_Market_Cap_Outstanding_Forward_Total_Revenue',
               'Diluted_Market_Cap_Exercisable_Forward_Total_Revenue',
               'Short_Interest_Ratio', 'Day_Open_Price', 'Day_Close_Price',
               'Day_High_Price', 'Day_Low_Price', 'VWAP',
               'Shares_Outstanding', 'Volume', 'FinalScore']

# Define prediction horizon (30 days ahead)
TARGET_WINDOW_DAYS = 30

# Initialize lists to store features, targets, and sample weights
X_list = []
y_list = []
sample_weights = []

# Iterate through each stock to prepare data
for ticker, df in stock_data.items():
    if len(df) < (TARGET_WINDOW_DAYS + 1):
        continue  # Skip stocks with insufficient data

    # Create target variable: percentage return over the next TARGET_WINDOW_DAYS
    df['Future_Close'] = df['Day_Close_Price'].shift(-TARGET_WINDOW_DAYS)
    df['Target_Return'] = ((df['Future_Close'] - df['Day_Close_Price']) / df['Day_Close_Price']) * 100
    df.dropna(inplace=True)

    # Features and target
    X = df[feature_cols].values
    y = df['Target_Return'].values

    # Assign weights: higher weight for higher FinalScore
    # Normalize FinalScore to [0.5, 1.5] for weighting purposes
    score = df['FinalScore'].iloc[0]  # Assuming FinalScore is constant per stock
    max_score = final_rankings['FinalScore'].max()
    weight = 1 + (score / max_score)  # Adjust as needed

    weights = np.full_like(y, weight, dtype=np.float32)

    X_list.append(X)
    y_list.append(y)
    sample_weights.append(weights)

# Combine all stocks' data
X = np.vstack(X_list)
y = np.hstack(y_list)
sample_weights = np.hstack(sample_weights)

print(f"Total samples after combining all stocks: {X.shape[0]}")

Total samples after combining all stocks: 754600


In [9]:
# %% [markdown]
# # 8. Define Helper Functions
#
# Define functions to check the existence of Optuna studies and to allocate investments based on expected profits.

# %%
# Function to determine if a study exists
def study_exists(study_name, storage_name):
    from optuna.storages import RDBStorage
    try:
        storage = RDBStorage(url=storage_name)
        storage.get_study_id_from_name(study_name)
        return True
    except KeyError:
        return False

# Function to allocate investment to the top-performing stock
def allocate_investment(profit_df, total_investment=30):
    """
    Allocates the total investment to the stock with the highest expected profit.

    Parameters:
    - profit_df (pd.DataFrame): DataFrame containing 'Stock', 'Expected_Profit'.
    - total_investment (float): Total amount to invest.

    Returns:
    - allocation (dict): Dictionary with 'Stock', 'Investment', 'Expected_Profit'.
    - total_profit (float): Total expected profit from the investment.
    """
    if profit_df.empty:
        print("No adequate stocks available for investment.")
        return None, 0.0

    # Sort the DataFrame by Expected Profit in descending order (higher is better)
    sorted_df = profit_df.sort_values(by='Expected_Profit', ascending=False).reset_index(drop=True)

    # Select the top stock
    top_stock = sorted_df.iloc[0]

    # Allocate the entire investment to the top stock
    allocation = {
        'Stock': top_stock['Stock'],
        'Investment': round(total_investment, 2),
        'Expected_Profit': round(top_stock['Expected_Profit'], 2)
    }

    # Total profit is the expected profit from the top stock
    total_profit = allocation['Expected_Profit']

    return allocation, total_profit

In [10]:
# %%
import pandas as pd
import numpy as np

# 1. Define the replacement function
def replace_NM_with_prev_or_zero(df):
    """
    Replace all occurrences of 'NM' in any column with the value from the row above in that column.
    If the row above is inaccessible, replace 'NM' with 0.0.

    Parameters:
    - df (pd.DataFrame): The input DataFrame to process.

    Returns:
    - pd.DataFrame: The processed DataFrame with 'NM' replaced appropriately.
    """
    df = df.copy()  # To avoid modifying the original DataFrame

    # Replace 'NM' with NaN to facilitate forward filling
    df.replace('NM', np.nan, inplace=True)

    # Forward fill to replace NaN with the value from the row above
    df.fillna(method='ffill', inplace=True)

    # For any remaining NaN (which were 'NM' in the first row), replace with 0.0
    df.fillna(0.0, inplace=True)

    # Convert all columns to numeric types where possible
    for col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='ignore')

    return df

# 2. Apply the function to all stocks in stock_data
for ticker, df in stock_data.items():
    stock_data[ticker] = replace_NM_with_prev_or_zero(df)
    print(f"Processed stock: {ticker}")

# 3. Verify the replacement
def verify_replacement(stock_data):
    """
    Verify that 'NM' has been replaced with previous row's value or 0.0 in all columns for all stocks.

    Parameters:
    - stock_data (dict): Dictionary containing stock tickers as keys and DataFrames as values.

    Returns:
    - None
    """
    for ticker, df in stock_data.items():
        # Check if 'NM' still exists in any column
        if (df.astype(str) == 'NM').any().any():
            print(f"'NM' still found in stock: {ticker}")
        else:
            print(f"No 'NM' values found in stock: {ticker}")

# Run the verification
verify_replacement(stock_data)

Processed stock: ANET
Processed stock: AXON
Processed stock: ANSS
Processed stock: AVGO
Processed stock: AMZN
Processed stock: AON
Processed stock: APD
Processed stock: AMGN
Processed stock: AMD
Processed stock: AMP
Processed stock: AKAM
Processed stock: AMAT
Processed stock: ALNY
Processed stock: ADP
Processed stock: AJG
Processed stock: AIZ
Processed stock: ADSK
Processed stock: A
Processed stock: ADBE
Processed stock: ADI
Processed stock: AXP
Processed stock: AAPL
Processed stock: ACGL
Processed stock: ABBV
Processed stock: BKNG
Processed stock: BK
Processed stock: BR
Processed stock: BAC
Processed stock: BKR
Processed stock: BLK
Processed stock: BXP
Processed stock: BBY
Processed stock: CRM
Processed stock: CRH
Processed stock: CSCO
Processed stock: CPRT
Processed stock: COR
Processed stock: COP
Processed stock: COST
Processed stock: CPB
Processed stock: CNH
Processed stock: COO
Processed stock: CMS
Processed stock: COF
Processed stock: CMI
Processed stock: CI
Processed stock: CMCS

In [11]:
# %% [markdown]
# # 9. Hyperparameter Tuning and Model Training
#
# Train CatBoost models for each stock using Optuna for hyperparameter tuning. Calculate R² and MAPE for each model, and ensure that progress is saved persistently.

# %%
# Define the number of trials for Optuna
N_TRIALS_CB = 1  # Adjust based on required accuracy and available computation time

# Initialize a dictionary to store best parameters
if os.path.exists(BEST_PARAMS_PATH):
    with open(BEST_PARAMS_PATH, 'rb') as f:
        best_params_dict = pickle.load(f)
else:
    best_params_dict = {}

# Initialize a dictionary to store forecasts
if os.path.exists(FORECASTS_PATH):
    with open(FORECASTS_PATH, 'rb') as f:
        stock_forecasts = pickle.load(f)
else:
    stock_forecasts = {}

# Initialize a list to store model performance
if os.path.exists(PERFORMANCE_PATH):
    with open(PERFORMANCE_PATH, 'rb') as f:
        model_performance = pickle.load(f)
else:
    model_performance = []

# Iterate through each stock to train models
for ticker in stock_data:
    # Skip if the stock has already been processed
    if ticker in stock_forecasts and 'CatBoost' in stock_forecasts[ticker]:
        print(f"Skipping CatBoost for {ticker} as it has already been processed.")
        continue

    print(f"\nProcessing CatBoost model for stock: {ticker}")
    df = stock_data[ticker]

    # Define features and target
    X_stock = df[feature_cols]
    y_stock = df['Target_Return']
    w_stock = sample_weights[:len(X_stock)]  # Assuming sample_weights are aligned

    # Define study name and storage
    study_name = f"cb_study_{ticker}"
    storage_name = f"sqlite:///{STUDIES_DIR}/cb_study_{ticker}.db"

    # Check if the study already exists
    if study_exists(study_name, storage_name):
        study = optuna.load_study(study_name=study_name, storage=storage_name)
        print(f"Loaded existing CatBoost study for {ticker}.")
    else:
        study = optuna.create_study(direction='minimize', study_name=study_name, storage=storage_name, load_if_exists=True, pruner=MedianPruner())
        print(f"Created new CatBoost study for {ticker}.")

    # Define the objective function within the loop to capture current X and y
    def objective_cb(trial, X=X_stock, y=y_stock, w=w_stock):
        param = {
            'iterations': trial.suggest_int('iterations', 500, 3000),
            'depth': trial.suggest_int('depth', 4, 12),
            'learning_rate': trial.suggest_float('learning_rate', 1e-4, 0.3, log=True),
            'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 15),
            'random_strength': trial.suggest_float('random_strength', 1, 10),
            'bagging_temperature': trial.suggest_float('bagging_temperature', 0, 10),
            'od_wait': 50,
            'random_seed': 42,
            'loss_function': 'RMSE',
            'verbose': False
        }

        model = CatBoostRegressor(**param)
        tscv = TimeSeriesSplit(n_splits=3)  # Reduced splits for faster tuning

        mse_scores = []
        r2_scores = []
        all_preds = []
        all_true = []

        for i, (train_index, test_index) in enumerate(tscv.split(X)):
            X_train_fold, X_val_fold = X.iloc[train_index], X.iloc[test_index]
            y_train_fold, y_val_fold = y.iloc[train_index], y.iloc[test_index]
            w_train_fold, w_val_fold = w[train_index], w[test_index]

            # Create Pool with sample weights
            train_pool = Pool(X_train_fold, y_train_fold, weight=w_train_fold)

            model.fit(train_pool, eval_set=(X_val_fold, y_val_fold), early_stopping_rounds=100, use_best_model=True)
            preds = model.predict(X_val_fold)
            mse = mean_squared_error(y_val_fold, preds)
            r2 = r2_score(y_val_fold, preds)
            mse_scores.append(mse)
            r2_scores.append(r2)
            all_preds.extend(preds)
            all_true.extend(y_val_fold)

            # Report intermediate objective value
            trial.report(mse, i+1)

            # Prune trial if not promising
            if trial.should_prune():
                raise TrialPruned()

        # Store the average R² and MAPE as user attributes
        trial.set_user_attr("r2", np.mean(r2_scores))
        trial.set_user_attr("mape", mean_absolute_percentage_error(all_true, all_preds))
        trial.set_user_attr("preds", all_preds)
        trial.set_user_attr("true", all_true)

        return np.mean(mse_scores)

    try:
        # Optimize hyperparameters using Optuna
        study.optimize(objective_cb, n_trials=N_TRIALS_CB, timeout=1800)  # 30 minutes timeout

        best_params = study.best_params
        print(f"Best params for {ticker} (CatBoost): {best_params}")

        # Retrieve the best trial
        best_trial = study.best_trial

        # Store best RMSE, R², and MAPE
        model_performance.append({
            'Stock': ticker,
            'Model': 'CatBoost',
            'RMSE': best_trial.value,  # Mean MSE across folds
            'R2': best_trial.user_attrs.get("r2", None),
            'MAPE': best_trial.user_attrs.get("mape", None)
        })

        print(f"Best RMSE for {ticker}: {best_trial.value}")
        print(f"Best R² for {ticker}: {best_trial.user_attrs.get('r2', None)}")
        print(f"Best MAPE for {ticker}: {best_trial.user_attrs.get('mape', None)}")

        # Save best parameters
        best_params_dict.setdefault(ticker, {})
        best_params_dict[ticker]['CatBoost'] = best_params

        # Train the best model on the entire dataset
        best_model = CatBoostRegressor(**best_params, loss_function='RMSE', random_seed=42, verbose=False)
        train_pool_full = Pool(X_stock, y_stock, weight=w_stock)
        best_model.fit(train_pool_full, eval_set=(X_stock, y_stock), use_best_model=True)

        # Forecasting: Predict the Target_Return on the last day
        latest_features = df[feature_cols].iloc[-1].values.reshape(1, -1)
        forecast_return = best_model.predict(latest_features)[0]

        # Store the forecast
        stock_forecasts.setdefault(ticker, {})
        stock_forecasts[ticker]['CatBoost'] = forecast_return

        print(f"Completed CatBoost for {ticker}. Forecast Return: {forecast_return:.4f}%")

    except TrialPruned:
        print(f"CatBoost trial for {ticker} was pruned.")
    except Exception as e:
        print(f"An error occurred while processing CatBoost for {ticker}: {e}")

    # Save the best parameters after each stock
    with open(BEST_PARAMS_PATH, 'wb') as f:
        pickle.dump(best_params_dict, f)

    # Save the forecasts and performance after each stock
    with open(FORECASTS_PATH, 'wb') as f:
        pickle.dump(stock_forecasts, f)

    with open(PERFORMANCE_PATH, 'wb') as f:
        pickle.dump(model_performance, f)

    print(f"Saved CatBoost parameters, forecasts, and performance data for {ticker}.")

Skipping CatBoost for ANET as it has already been processed.
Skipping CatBoost for AXON as it has already been processed.
Skipping CatBoost for ANSS as it has already been processed.
Skipping CatBoost for AVGO as it has already been processed.
Skipping CatBoost for AMZN as it has already been processed.
Skipping CatBoost for AON as it has already been processed.
Skipping CatBoost for APD as it has already been processed.
Skipping CatBoost for AMGN as it has already been processed.
Skipping CatBoost for AMD as it has already been processed.
Skipping CatBoost for AMP as it has already been processed.
Skipping CatBoost for AKAM as it has already been processed.
Skipping CatBoost for AMAT as it has already been processed.
Skipping CatBoost for ALNY as it has already been processed.
Skipping CatBoost for ADP as it has already been processed.
Skipping CatBoost for AJG as it has already been processed.
Skipping CatBoost for AIZ as it has already been processed.
Skipping CatBoost for ADSK as i

[I 2024-12-15 06:03:30,023] Trial 42 pruned. 


Best params for EQT (CatBoost): {'iterations': 2832, 'depth': 10, 'learning_rate': 0.0004155450736528692, 'l2_leaf_reg': 5.842041906231419, 'random_strength': 6.8902580443155355, 'bagging_temperature': 4.543213160931296}
Best RMSE for EQT: 301.22141673573634
Best R² for EQT: -0.06455149728984366
Best MAPE for EQT: 33997106397903.574
Completed CatBoost for EQT. Forecast Return: 2.9590%
Saved CatBoost parameters, forecasts, and performance data for EQT.

Processing CatBoost model for stock: FDS
Loaded existing CatBoost study for FDS.


[I 2024-12-15 06:06:32,331] Trial 5 pruned. 


Best params for FDS (CatBoost): {'iterations': 2516, 'depth': 12, 'learning_rate': 0.0005686748226746227, 'l2_leaf_reg': 4.168961569207648, 'random_strength': 5.557849701923356, 'bagging_temperature': 7.486195495421928}
Best RMSE for FDS: 43.28806984052019
Best R² for FDS: 0.034240280273830304
Best MAPE for FDS: 40392579844.420105
Completed CatBoost for FDS. Forecast Return: 0.2080%
Saved CatBoost parameters, forecasts, and performance data for FDS.

Processing CatBoost model for stock: EXPD
Loaded existing CatBoost study for EXPD.


...

Best params for UNH (CatBoost): {'iterations': 2185, 'depth': 8, 'learning_rate': 0.042427063998839436, 'l2_leaf_reg': 5.185500800274568, 'random_strength': 1.51573850265076, 'bagging_temperature': 0.09918519314308405}
Best RMSE for UNH: 40.62019745516478
Best R² for UNH: 0.048598323561009425
Best MAPE for UNH: 1.880644406950555
Completed CatBoost for UNH. Forecast Return: -9.3944%
Saved CatBoost parameters, forecasts, and performance data for UNH.

Processing CatBoost model for stock: WTW
Loaded existing CatBoost study for WTW.


[I 2024-12-15 09:02:30,558] Trial 35 finished with value: 35.44211848394016 and parameters: {'iterations': 2142, 'depth': 6, 'learning_rate': 0.1105434489216777, 'l2_leaf_reg': 7.7246946762593, 'random_strength': 1.0154792166527817, 'bagging_temperature': 8.076262362107837}. Best is trial 35 with value: 35.44211848394016.


Best params for WTW (CatBoost): {'iterations': 2142, 'depth': 6, 'learning_rate': 0.1105434489216777, 'l2_leaf_reg': 7.7246946762593, 'random_strength': 1.0154792166527817, 'bagging_temperature': 8.076262362107837}
Best RMSE for WTW: 35.44211848394016
Best R² for WTW: 0.07292062305504869
Best MAPE for WTW: 2058854443999.0833
Completed CatBoost for WTW. Forecast Return: -0.3690%
Saved CatBoost parameters, forecasts, and performance data for WTW.

Processing CatBoost model for stock: STE
Loaded existing CatBoost study for STE.


[I 2024-12-15 09:02:48,287] Trial 35 pruned. 


Best params for STE (CatBoost): {'iterations': 2541, 'depth': 11, 'learning_rate': 0.1386330481311653, 'l2_leaf_reg': 14.987171407294884, 'random_strength': 1.6725438850257386, 'bagging_temperature': 1.5602256944131732}
Best RMSE for STE: 46.16534433376281
Best R² for STE: 0.03237896969023857
Best MAPE for STE: 1.8004223547524132
Completed CatBoost for STE. Forecast Return: -1.7945%
Saved CatBoost parameters, forecasts, and performance data for STE.


In [12]:
#PPG and done

In [13]:
# %% [markdown]
# # 10. Calculate Investment Profits and Allocate Funds
#
# Calculate expected profits based on model forecasts and allocate investment funds accordingly. Ensure that only adequate stocks are considered.

# %%
# Identify adequate and inadequate stocks based on performance data
all_stocks = list(stock_data.keys())

adequate_stocks = []
inadequate_stocks = []

for stock in all_stocks:
    # Check if the stock has a forecast and RMSE
    if stock in stock_forecasts and 'CatBoost' in stock_forecasts[stock]:
        # Further check if RMSE and R2 are not None (indicating successful processing)
        performance = next((item for item in model_performance if item['Stock'] == stock and item['Model'] == 'CatBoost'), None)
        if performance and performance['RMSE'] is not None and performance['R2'] is not None:
            adequate_stocks.append(stock)
        else:
            inadequate_stocks.append(stock)
    else:
        inadequate_stocks.append(stock)

print(f"Total Stocks: {len(all_stocks)}")
print(f"Number of Adequate Stocks: {len(adequate_stocks)}")
print(f"Number of Inadequate Stocks: {len(inadequate_stocks)}")
print("\nList of Inadequate Stocks:")
print(inadequate_stocks)

Total Stocks: 210
Number of Adequate Stocks: 204
Number of Inadequate Stocks: 6

List of Inadequate Stocks:
['FTV', 'HPE', 'GDDY', 'KHC', 'TWLO', 'TEAM']


In [14]:
# %%
# Initialize a list to store profit calculations
profit_list = []

for stock in adequate_stocks:
    try:
        # Retrieve the forecasted return
        forecast_return = stock_forecasts[stock]['CatBoost']

        # Get the current Close price (last available in the DataFrame)
        current_price = stock_data[stock]['Day_Close_Price'].iloc[-1]

        # Calculate the expected return in GBP
        expected_profit = (forecast_return / 100) * current_price

        # Append a dictionary to the list
        profit_list.append({
            'Stock': stock,
            'Current_Price': round(current_price, 2),
            'Forecasted_Return': round(forecast_return, 2),
            'Expected_Profit': round(expected_profit, 2)
        })
    except Exception as e:
        print(f"Error processing stock {stock}: {e}")
        # Optionally, log the error or mark the stock as inadequate
        inadequate_stocks.append(stock)

# Convert the list of dictionaries to a DataFrame
profit_df = pd.DataFrame(profit_list)

# Sort the DataFrame by Expected Profit in descending order (higher profit is better)
profit_df.sort_values(by='Expected_Profit', ascending=False, inplace=True)

# Reset index
profit_df.reset_index(drop=True, inplace=True)

# Display the profit DataFrame
profit_df

,Stock,Current_Price,Forecasted_Return,Expected_Profit
0,BKNG,4943.27,5.45,269.17
1,FICO,2332.29,7.52,175.31
2,MSTR,270.42,43.95,118.86
3,NFLX,795.04,14.72,117.05
4,NOW,1008.08,11.48,115.73
...,...,...,...,...
199,MTD,1310.03,-3.69,-48.35
200,SNPS,561.55,-8.69,-48.78
201,AMGN,325.28,-15.37,-50.00
202,NOC,528.99,-9.67,-51.15


In [15]:
# %%
# Total Investment
total_investment = 30.0  # in GBP

# Calculate the total expected profit of all adequate stocks
total_expected_profit = profit_df['Expected_Profit'].sum()

# Calculate the weight (allocation percentage) for each stock based on its expected profit
# Since higher Expected_Profit is better, allocate more to stocks with higher Expected_Profit
# Normalize Expected_Profit to sum to 1 for allocation
if total_expected_profit == 0:
    profit_df['Weight'] = 1 / len(profit_df)
else:
    profit_df['Weight'] = profit_df['Expected_Profit'] / total_expected_profit

# Calculate the investment allocation for each stock
profit_df['Investment'] = profit_df['Weight'] * total_investment

# Calculate the number of shares for each stock
profit_df['Number_of_Shares'] = profit_df['Investment'] / profit_df['Current_Price']

# Calculate the expected profit for the allocated investment
profit_df['Allocated_Profit'] = profit_df['Number_of_Shares'] * (profit_df['Forecasted_Return'])

# Round the numerical columns for better readability
profit_df[['Investment', 'Number_of_Shares', 'Allocated_Profit']] = profit_df[['Investment', 'Number_of_Shares', 'Allocated_Profit']].round(2)

# Display the updated DataFrame with allocations
profit_df[['Stock', 'Current_Price', 'Forecasted_Return', 'Expected_Profit', 'Investment', 'Number_of_Shares', 'Allocated_Profit']]

,Stock,Current_Price,Forecasted_Return,Expected_Profit,Investment,Number_of_Shares,Allocated_Profit
0,BKNG,4943.27,5.45,269.17,9.38,0.00,0.01
1,FICO,2332.29,7.52,175.31,6.11,0.00,0.02
2,MSTR,270.42,43.95,118.86,4.14,0.02,0.67
3,NFLX,795.04,14.72,117.05,4.08,0.01,0.08
4,NOW,1008.08,11.48,115.73,4.03,0.00,0.05
...,...,...,...,...,...,...,...
199,MTD,1310.03,-3.69,-48.35,-1.68,-0.00,0.00
200,SNPS,561.55,-8.69,-48.78,-1.70,-0.00,0.03
201,AMGN,325.28,-15.37,-50.00,-1.74,-0.01,0.08
202,NOC,528.99,-9.67,-51.15,-1.78,-0.00,0.03


In [16]:
# %%
# Final Ranked List with Investment Strategy
final_ranking = profit_df[['Stock', 'Current_Price', 'Forecasted_Return', 'Expected_Profit', 'Investment', 'Number_of_Shares', 'Allocated_Profit']].copy()

# Calculate the total allocated investment and total expected profit
total_allocated_investment = final_ranking['Investment'].sum()
total_allocated_profit = final_ranking['Allocated_Profit'].sum()

# Display the final ranking
print("Final Ranked List of Stocks Based on Expected Profit:")
display(final_ranking)

print(f"Total Allocated Investment: £{round(total_allocated_investment, 2)}")
print(f"Total Expected Profit: £{round(total_allocated_profit, 2)}")

Final Ranked List of Stocks Based on Expected Profit:


,Stock,Current_Price,Forecasted_Return,Expected_Profit,Investment,Number_of_Shares,Allocated_Profit
0,BKNG,4943.27,5.45,269.17,9.38,0.00,0.01
1,FICO,2332.29,7.52,175.31,6.11,0.00,0.02
2,MSTR,270.42,43.95,118.86,4.14,0.02,0.67
3,NFLX,795.04,14.72,117.05,4.08,0.01,0.08
4,NOW,1008.08,11.48,115.73,4.03,0.00,0.05
...,...,...,...,...,...,...,...
199,MTD,1310.03,-3.69,-48.35,-1.68,-0.00,0.00
200,SNPS,561.55,-8.69,-48.78,-1.70,-0.00,0.03
201,AMGN,325.28,-15.37,-50.00,-1.74,-0.01,0.08
202,NOC,528.99,-9.67,-51.15,-1.78,-0.00,0.03


Total Allocated Investment: £30.0
Total Expected Profit: £3.11


In [17]:
# %% [markdown]
# # 11. Display Inadequate Stocks
#
# Show a list of stocks that were excluded from the analysis due to the absence of `FinalScore` or failed model evaluations.

# %%
# Display Inadequate Stocks
if inadequate_stocks:
    inadequate_df = pd.DataFrame({'Inadequate Stocks': inadequate_stocks})
    print("List of Inadequate Stocks (Excluded from Analysis):")
    display(inadequate_df)
else:
    print("No Inadequate Stocks found.")

List of Inadequate Stocks (Excluded from Analysis):


,Inadequate Stocks
0,FTV
1,HPE
2,GDDY
3,KHC
4,TWLO
5,TEAM


In [18]:
# %% [markdown]
# # 12. Summary of Investment Strategy
#
# Provide a summarized view of the investment strategy, including total investment, expected profit, portfolio allocation, and a list of excluded stocks.

# %%
# Summary
print("### Investment Strategy Summary ###\n")

print(f"**Total Investment Available:** £{total_investment}\n")
print(f"**Total Expected Profit:** £{round(total_allocated_profit, 2)}\n")
print("**Portfolio Allocation:**")
display(final_ranking[['Stock', 'Investment', 'Number_of_Shares', 'Allocated_Profit']])

if inadequate_stocks:
    print("\n**Inadequate Stocks (Excluded from Portfolio):**")
    display(inadequate_df)
else:
    print("\n**All stocks were adequate for the analysis.**")

### Investment Strategy Summary ###

**Total Investment Available:** £30.0

**Total Expected Profit:** £3.11

**Portfolio Allocation:**


,Stock,Investment,Number_of_Shares,Allocated_Profit
0,BKNG,9.38,0.00,0.01
1,FICO,6.11,0.00,0.02
2,MSTR,4.14,0.02,0.67
3,NFLX,4.08,0.01,0.08
4,NOW,4.03,0.00,0.05
...,...,...,...,...
199,MTD,-1.68,-0.00,0.00
200,SNPS,-1.70,-0.00,0.03
201,AMGN,-1.74,-0.01,0.08
202,NOC,-1.78,-0.00,0.03



**Inadequate Stocks (Excluded from Portfolio):**


,Inadequate Stocks
0,FTV
1,HPE
2,GDDY
3,KHC
4,TWLO
5,TEAM


In [19]:
# %% [markdown]
# # 13. Visualize R² and MAPE Scores
#
# Create visualizations to understand the distribution and relationship between R² and MAPE scores across all models.

# %%
# Distribution of R² Scores
plt.figure(figsize=(10,6))
sns.histplot(performance_df['R2'].dropna(), bins=30, kde=True, color='skyblue')
plt.title('Distribution of R² Scores for CatBoost Models')
plt.xlabel('R² Score')
plt.ylabel('Frequency')
plt.show()

# Distribution of MAPE
plt.figure(figsize=(10,6))
sns.histplot(performance_df['MAPE'].dropna(), bins=30, kde=True, color='salmon')
plt.title('Distribution of MAPE for CatBoost Models')
plt.xlabel('MAPE (%)')
plt.ylabel('Frequency')
plt.show()

# Scatter Plot of R² vs MAPE
plt.figure(figsize=(10,6))
sns.scatterplot(data=performance_df, x='R2', y='MAPE', hue='R2', palette='coolwarm', alpha=0.7)
plt.title('R² vs MAPE for CatBoost Models')
plt.xlabel('R² Score')
plt.ylabel('MAPE (%)')
plt.legend(title='R²')
plt.show()

# Boxplots for R² and MAPE
plt.figure(figsize=(14,6))

# Boxplot for R²
plt.subplot(1,2,1)
sns.boxplot(y=performance_df['R2'], color='lightblue')
plt.title('Boxplot of R² Scores')
plt.ylabel('R² Score')

# Boxplot for MAPE
plt.subplot(1,2,2)
sns.boxplot(y=performance_df['MAPE'], color='lightgreen')
plt.title('Boxplot of MAPE')
plt.ylabel('MAPE (%)')

plt.tight_layout()
plt.show()

# Summary Statistics for R² and MAPE
print("### Summary Statistics ###\n")
print(performance_df[['R2', 'MAPE']].describe())

NameError: name 'performance_df' is not defined

<Figure size 1000x600 with 0 Axes>

In [ ]:
# %% [markdown]
# # 14. Plot Predicted vs. Actual Returns for Top Performing Stocks
#
# Visualize the accuracy of your models by plotting predicted vs. actual returns for top-performing stocks based on R² scores.

# %%
# Select top-performing stocks based on R²
TOP_N_STOCKS = 5  # Adjust as needed
top_performing_stocks = performance_df.sort_values(by='R2', ascending=False).head(TOP_N_STOCKS)['Stock'].tolist()
print(f"Top Performing Stocks based on R²: {top_performing_stocks}")

# Plot Predicted vs Actual for top-performing stocks
for stock in top_performing_stocks:
    # Load the corresponding study
    study_name = f"cb_study_{stock}"
    storage_name = f"sqlite:///{STUDIES_DIR}/cb_study_{stock}.db"
    study_db_path = os.path.join(DATA_PATH, 'optuna_studies', f"cb_study_{stock}.db")

    if not os.path.exists(study_db_path):
        print(f"Study database for {stock} not found. Skipping plot.")
        continue

    try:
        study = optuna.load_study(study_name=study_name, storage=storage_name)
        best_trial = study.best_trial
        preds = best_trial.user_attrs.get("preds", [])
        true = best_trial.user_attrs.get("true", [])

        if preds and true:
            plt.figure(figsize=(8,6))
            sns.scatterplot(x=true, y=preds, alpha=0.6)
            plt.plot([min(true), max(true)], [min(true), max(true)], 'r--')  # Diagonal line
            plt.title(f'Predicted vs. Actual Returns for {stock}')
            plt.xlabel('Actual Returns (%)')
            plt.ylabel('Predicted Returns (%)')
            plt.show()
        else:
            print(f"Predictions or true values not found for {stock}. Skipping plot.")
    except Exception as e:
        print(f"Error loading study for {stock}: {e}")